# More infomation about this dataset: 
### https://www.kaggle.com/datasets/sukhmandeepsinghbrar/car-price-prediction-dataset

### Nhập thư viện cần dùng

In [53]:
import pandas as pd
import numpy as np

### Đọc dữ liệu file csv

In [56]:
ds = pd.read_csv("car_price.csv")
ds

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage(km/ltr/kg),engine,max_power,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.40,1248.0,74,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14,1498.0,103.52,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.70,1497.0,78,5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.00,1396.0,90,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.10,1298.0,88.2,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
8123,Hyundai i20 Magna,2013,320000,110000,Petrol,Individual,Manual,First Owner,18.50,1197.0,82.85,5.0
8124,Hyundai Verna CRDi SX,2007,135000,119000,Diesel,Individual,Manual,Fourth & Above Owner,16.80,1493.0,110,5.0
8125,Maruti Swift Dzire ZDi,2009,382000,120000,Diesel,Individual,Manual,First Owner,19.30,1248.0,73.9,5.0
8126,Tata Indigo CR4,2013,290000,25000,Diesel,Individual,Manual,First Owner,23.57,1396.0,70,5.0


Một số đặc điểm của dataset

In [59]:
ds.info()
ds.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                8128 non-null   object 
 1   year                8128 non-null   int64  
 2   selling_price       8128 non-null   int64  
 3   km_driven           8128 non-null   int64  
 4   fuel                8128 non-null   object 
 5   seller_type         8128 non-null   object 
 6   transmission        8128 non-null   object 
 7   owner               8128 non-null   object 
 8   mileage(km/ltr/kg)  7907 non-null   float64
 9   engine              7907 non-null   float64
 10  max_power           7913 non-null   object 
 11  seats               7907 non-null   float64
dtypes: float64(3), int64(3), object(6)
memory usage: 762.1+ KB


Index(['name', 'year', 'selling_price', 'km_driven', 'fuel', 'seller_type',
       'transmission', 'owner', 'mileage(km/ltr/kg)', 'engine', 'max_power',
       'seats'],
      dtype='object')

Chuyển cột max_power từ kiểu object sang float64 để xử lý

In [62]:
ds["max_power"] = pd.to_numeric(ds['max_power'],errors = 'coerce') # convert object to float64 in 'max_power' column

Tìm giá trị bị thiếu trên các cột

In [65]:
for i in range(len(ds.columns)):
    missing_data = ds[ds.columns[i]].isna().sum()
    percent = missing_data / len(ds) * 100
    print('%d Column name: %s\t\t\tmissing entries: %d\t\t\tpercentage: %.2f' % (i, ds.columns[i], missing_data, percent))

0 Column name: name			missing entries: 0			percentage: 0.00
1 Column name: year			missing entries: 0			percentage: 0.00
2 Column name: selling_price			missing entries: 0			percentage: 0.00
3 Column name: km_driven			missing entries: 0			percentage: 0.00
4 Column name: fuel			missing entries: 0			percentage: 0.00
5 Column name: seller_type			missing entries: 0			percentage: 0.00
6 Column name: transmission			missing entries: 0			percentage: 0.00
7 Column name: owner			missing entries: 0			percentage: 0.00
8 Column name: mileage(km/ltr/kg)			missing entries: 221			percentage: 2.72
9 Column name: engine			missing entries: 221			percentage: 2.72
10 Column name: max_power			missing entries: 216			percentage: 2.66
11 Column name: seats			missing entries: 221			percentage: 2.72


Các đặc tính được chọn lọc

In [68]:
ft=['year', 'km_driven', 'fuel', 'transmission', 'owner', 'mileage(km/ltr/kg)', 'engine', 'max_power', 'seats']

In [70]:
# chuyển dataframe thành mảng numpy để xử lý
X = ds[ft].values # tập X cho dữ liệu phụ thuộc
y = ds['selling_price'].values # tâp y cho dữ liệu không phụ thuộc

In [72]:
print(X)
print(y)

[[2014 145500 'Diesel' ... 1248.0 74.0 5.0]
 [2014 120000 'Diesel' ... 1498.0 103.52 5.0]
 [2006 140000 'Petrol' ... 1497.0 78.0 5.0]
 ...
 [2009 120000 'Diesel' ... 1248.0 73.9 5.0]
 [2013 25000 'Diesel' ... 1396.0 70.0 5.0]
 [2013 25000 'Diesel' ... 1396.0 70.0 5.0]]
[450000 370000 158000 ... 382000 290000 290000]


Xử lý giá trị bị thiếu bằng phương pháp trung bình

In [75]:
for k in range(5,9): # xét các cột có giá trị nan
    a = 0
    b = 0
    for i in X[:,k]: # duyệt mảng X[:,k]
        if np.isnan(i):
            continue
        a += 1 # số lượng phần tử khác nan
        b = b + i # tổng các phần tử khác nan
    for i in range(len(X[:,k])):
        if np.isnan(X[i,k]):
            X[i,k] = b/a # trung bình cộng của các phần tử khác nan

In [77]:
print(X)

[[2014 145500 'Diesel' ... 1248.0 74.0 5.0]
 [2014 120000 'Diesel' ... 1498.0 103.52 5.0]
 [2006 140000 'Petrol' ... 1497.0 78.0 5.0]
 ...
 [2009 120000 'Diesel' ... 1248.0 73.9 5.0]
 [2013 25000 'Diesel' ... 1396.0 70.0 5.0]
 [2013 25000 'Diesel' ... 1396.0 70.0 5.0]]


Mã hoá kiểu kí tự thành số

In [80]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
#output of fit_transform of Label Encoder is already a Numpy Array
a = le.fit_transform(X[:,2])
b = le.fit_transform(X[:,3])
c = le.fit_transform(X[:,4])

In [82]:
X[:,2] = a
X[:,3] = b
X[:,4] = c

In [84]:
# Co kéo dữ liệu
a = X[:,0].max()
b = X[:,1].max()
c = X[:,5].max()
d = X[:,6].max()
e = X[:,7].max()

In [86]:
for i in range(len(X[:,0])):
    X[i,0] = X[i,0]/a
    X[i,1] = X[i,1]/b
    X[i,5] = X[i,5]/c
    X[i,6] = X[i,6]/d
    X[i,7] = X[i,7]/e

In [88]:
print(X)

[[0.997029702970297 0.061640606035187256 1 ... 0.3462819089900111 0.185
  5.0]
 [0.997029702970297 0.0508376132248967 1 ... 0.4156492785793563 0.2588
  5.0]
 [0.9930693069306931 0.05931054876237949 3 ... 0.4153718091009989 0.195
  5.0]
 ...
 [0.9945544554455445 0.0508376132248967 1 ... 0.3462819089900111
  0.18475000000000003 5.0]
 [0.9965346534653465 0.01059116942185348 1 ... 0.3873473917869034 0.175
  5.0]
 [0.9965346534653465 0.01059116942185348 1 ... 0.3873473917869034 0.175
  5.0]]


Chia bộ dữ liệu train và test

In [91]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [93]:
print(X_train)
print(X_test)
print(y_train)
print(y_test)


[[0.997029702970297 0.04236467768741392 1 ... 0.4156492785793563 0.2465
  5.0]
 [0.998019801980198 0.0508376132248967 1 ... 0.3462819089900111 0.185 5.0]
 [0.996039603960396 0.04236467768741392 1 ... 0.4053829078801332 0.195
  5.0]
 ...
 [0.9965346534653465 0.029655274381189745 3 ... 0.22586015538290788 0.138
  5.0]
 [0.995049504950495 0.04660114545615531 1 ... 0.3462819089900111
  0.18475000000000003 5.0]
 [0.998019801980198 0.027856893813359023 3 ... 0.3321309655937847 0.20775
  5.0]]
[[0.9925742574257426 0.03389174214993114 1 ... 0.7000554938956715 0.1575
  7.0]
 [0.9985148514851485 0.019064104959336265 1 ... 0.6046059933407325
  0.38465000000000005 7.0]
 [0.9995049504950495 0.02541880661244835 1 ... 0.3873473917869034
  0.22190000000000001 5.0]
 ...
 [0.9945544554455445 0.023512396116514726 3 ... 0.2769145394006659
  0.16774999999999998 5.0]
 [0.9975247524752475 0.006354701653112088 1 ... 0.3462819089900111
  0.22125 5.0]
 [0.999009900990099 0.01059116942185348 3 ... 0.405660377358

### Training Model Random Forest

In [96]:
from sklearn.ensemble import RandomForestRegressor
CP_model = RandomForestRegressor(random_state = 1)
CP_model.fit(X_train, y_train)

RandomForestRegressor(random_state=1)

In [104]:
CP_model_preds = CP_model.predict(X_test)
pd.DataFrame(CP_model_preds, y_test).to_csv("du_doan_gia_xe.csv")